In [1]:
imageSize = [299, 299]
#trainPath = "input/diabetic-retinopathy-level-detection/preprocessed dataset/preprocessed dataset/training"
#testPath = "input/diabetic-retinopathy-level-detection/preprocessed dataset/preprocessed dataset/testing"
trainPath = "../input/diabetic-retinopathy-level-detection/preprocessed dataset/preprocessed dataset/training"
testPath = "../input/diabetic-retinopathy-level-detection/preprocessed dataset/preprocessed dataset/testing"

In [2]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
type(train_datagen)

keras.preprocessing.image.ImageDataGenerator

In [4]:
training_set =train_datagen.flow_from_directory(trainPath, target_size = (299,299),batch_size = 32, class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(testPath, target_size = (299,299),batch_size = 32, class_mode = 'categorical')

Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.


In [5]:
#xception = Xception(input_shape = imageSize + [3], weights = 'imagenet', include_top = False)
#xception = keras.models.load_model('../input/pretrained-model/xception_weights_tf_dim_ordering_tf_kernels_notop')
#xception = load_model('../input/xceptionmodelfile/xception_weights_tf_dim_ordering_tf_kernels_notop.h5')
xception = load_model('../input/xceptionsavedmodel3/assets')

2022-11-03 02:28:24.711877: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-11-03 02:28:25.940459: E tensorflow/core/framework/node_def_util.cc:629] NodeDef mentions attribute validate_shape which is not in the op definition: Op<name=AssignVariableOp; signature=resource:resource, value:dtype -> ; attr=dtype:type; is_stateful=true> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node AssignNewValue}}


In [6]:
#don't train existing weights
for layer in xception.layers:
  layer.trainable = False

In [7]:
#our layers - you can add more if you want
x = Flatten()(xception.output)

In [8]:
prediction = Dense(5, activation = 'softmax')(x)

In [9]:
#create a model object
model = Model(inputs = xception.input, outputs = prediction)

In [10]:
#view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [11]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [12]:
#fit the model
r = model.fit_generator(training_set, validation_data = test_set, epochs = 100, steps_per_epoch = len(training_set)//32, validation_steps = len(test_set)//32)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-11-03 02:28:38.162269: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
3/3 [==============================] - 39s 12s/step - loss: 6.7503 - accuracy: 0.4479
Epoch 2/100
3/3 [==============================] - 30s 10s/step - loss: 19.5675 - accuracy: 0.5208
Epoch 3/100
3/3 [==============================] - 29s 8s/step - loss: 9.9797 - accuracy: 0.5312
Epoch 4/100
3/3 [==============================] - 29s 9s/step - loss: 5.7716 - accuracy: 0.6042
Epoch 5/100
3/3 [==============================] - 33s 10s/step - loss: 6.7093 - accuracy: 0.6875
Epoch 6/100
3/3 [==============================] - 29s 9s/step - loss: 9.5670 - accuracy: 0.5729
Epoch 7/100
3/3 [==============================] - 30s 9s/step - loss: 5.6115 - accuracy: 0.6042
Epoch 8/100
3/3 [==============================] - 30s 9s/step - loss: 3.7252 - accuracy: 0.6354
Epoch 9/100
3/3 [==============================] - 30s 9s/step - loss: 3.0985 - accuracy: 0.6146
Epoch 10/100
3/3 [==============================] - 28s 8s/step - loss: 3.9273 - accuracy: 0.6458
Epoch 11/100
3/3 [=======